# DEFINICIÓN DE PARÁMETROS

In [ ]:
#######################################################################################################################

# Ruta hacia los CSV del dataset
train_file_relative = './data/700Train.csv'
validation_file_relative = './data/700Validation.csv'
test_file_relative = './data/700Test.csv'

# Ruta al directorio de imágenes con máscara aplicada
masked_dir_relative = '../local/output_masked'
edited_dir_relative = '../local/output_edited'

model_dir_relative = "./models"

# Ruta relativa hacia la librería ai4eutils
ai4eutils_relative = "./repos/ai4eutils"

# Ruta relativa hacia la librería CameraTraps
CameraTraps_relative = "./repos/CameraTraps"

#######################################################################################################################

# DEFINICIÓN PARÁMETROS CNN

In [ ]:
#######################################################################################################################

EPOCHS = 50
LEARNING_RATE = 1e-4
BATCH_SIZE = 16

#######################################################################################################################

In [ ]:
import os
import platform

import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow import keras

from modules.dataset_utils import DatasetUtils

home = os.path.expanduser("~")

ai4utils = os.path.abspath(ai4eutils_relative)
CameraTraps = os.path.abspath(CameraTraps_relative)
train_file = os.path.abspath(train_file_relative)
validation_file = os.path.abspath(validation_file_relative)
test_file = os.path.abspath(test_file_relative)
masked_dir = os.path.abspath(masked_dir_relative)
edited_dir = os.path.abspath(edited_dir_relative)
model_dir = os.path.abspath(model_dir_relative)

try:
    os.environ['PYTHONPATH']
except KeyError:
    os.environ['PYTHONPATH'] = ""
if platform.system() == 'Windows':
    os.environ['PYTHONPATH'] += (";" + ai4utils)
    os.environ['PYTHONPATH'] += (";" + CameraTraps)
else:
    os.environ['PYTHONPATH'] += (":" + ai4utils)
    os.environ['PYTHONPATH'] += (":" + CameraTraps)

print('==============================================================================================================')
print('PYTHONPATH: ' + os.environ['PYTHONPATH'])
print('')
print('ai4eutils PATH: ' + '\t\t' + ai4utils)
print('CameraTraps PATH: ' + '\t\t' + CameraTraps)
print('Train CSV PATH: ' + '\t\t' + train_file)
print('Validation CSV PATH: ' + '\t' + validation_file)
print('Test CSV PATH: ' + '\t\t\t' + test_file)
print('masked PATH: ' + '\t\t\t' + masked_dir)
print('edited PATH: ' + '\t\t\t' + edited_dir)
print('models PATH: ' + '\t\t\t' + model_dir)
print('==============================================================================================================')

# AlexNet architecture with image size 448x448

In [ ]:
# =====================================================================================================================
# Dataset with masked image
file_path, labels = DatasetUtils.load_dataset(train_file, masked_dir)
TRAIN_BUFFER = len(labels)
train_dataset_masked = tf.data.Dataset.from_tensor_slices((file_path, labels))
train_dataset_masked = train_dataset_masked.map(DatasetUtils.load_image).map(DatasetUtils.normalize_images).map(
    DatasetUtils.resize_448).shuffle(TRAIN_BUFFER).batch(BATCH_SIZE)

file_path, labels = DatasetUtils.load_dataset(validation_file, masked_dir)
VALIDATION_BUFFER = len(labels)
validation_dataset_masked = tf.data.Dataset.from_tensor_slices((file_path, labels))
validation_dataset_masked = validation_dataset_masked.map(DatasetUtils.load_image).map(
    DatasetUtils.normalize_images).map(DatasetUtils.resize_448).shuffle(VALIDATION_BUFFER).batch(BATCH_SIZE)

file_path, labels = DatasetUtils.load_dataset(test_file, masked_dir)
TEST_BUFFER = len(labels)
test_dataset_masked = tf.data.Dataset.from_tensor_slices((file_path, labels))
test_dataset_masked = test_dataset_masked.map(DatasetUtils.load_image).map(DatasetUtils.normalize_images).map(
    DatasetUtils.resize_448).shuffle(TEST_BUFFER).batch(BATCH_SIZE)

# =====================================================================================================================
# Dataset with edited image
file_path, labels = DatasetUtils.load_dataset(train_file, edited_dir)
TRAIN_BUFFER = len(labels)
train_dataset_edited = tf.data.Dataset.from_tensor_slices((file_path, labels))
train_dataset_edited = train_dataset_edited.map(DatasetUtils.load_image).map(DatasetUtils.normalize_images).map(
    DatasetUtils.resize_448).shuffle(TRAIN_BUFFER).batch(BATCH_SIZE)

file_path, labels = DatasetUtils.load_dataset(validation_file, edited_dir)
VALIDATION_BUFFER = len(labels)
validation_dataset_edited = tf.data.Dataset.from_tensor_slices((file_path, labels))
validation_dataset_edited = validation_dataset_edited.map(DatasetUtils.load_image).map(
    DatasetUtils.normalize_images).map(DatasetUtils.resize_448).shuffle(VALIDATION_BUFFER).batch(BATCH_SIZE)

file_path, labels = DatasetUtils.load_dataset(test_file, edited_dir)
TEST_BUFFER = len(labels)
test_dataset_edited = tf.data.Dataset.from_tensor_slices((file_path, labels))
test_dataset_edited = test_dataset_edited.map(DatasetUtils.load_image).map(DatasetUtils.normalize_images).map(
    DatasetUtils.resize_448).shuffle(TEST_BUFFER).batch(BATCH_SIZE)

In [ ]:
AlexNet_masked = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=(448, 448, 3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),

    keras.layers.Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),

    keras.layers.Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),
    keras.layers.Flatten(),

    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),

    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),

    keras.layers.Dense(2, activation='sigmoid')
])

AlexNet_masked.summary()
AlexNet_masked.compile(optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
                       loss=[keras.losses.BinaryCrossentropy()], metrics=['accuracy'])

In [ ]:
AlexNet_edited = tf.keras.models.clone_model(AlexNet_masked, input_tensors=None, clone_function=None)
AlexNet_edited.summary()
AlexNet_edited.compile(optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
                       loss=[keras.losses.BinaryCrossentropy()], metrics=['accuracy'])

In [ ]:
'''
hist = AlexNet_masked.fit(train_dataset_masked, epochs=EPOCHS, validation_data=validation_dataset_masked,
                          batch_size=BATCH_SIZE)
results = AlexNet_masked.evaluate(test_dataset_masked, verbose=1)

fig = plt.figure(figsize=(15, 5))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

fig.suptitle('AlexNet masked images 448x448')
ax1.plot(hist.history['loss'], color='blue', label='loss', )
ax1.plot(hist.history['val_loss'], color='green', label='val_loss')
ax2.plot(hist.history['accuracy'], color='orange', label='accuracy')
ax2.plot(hist.history['val_accuracy'], color='red', label='val_accuracy')

plt.legend(loc="upper left")
plt.show()
'''

In [ ]:
hist = AlexNet_edited.fit(train_dataset_edited, epochs=EPOCHS, validation_data=validation_dataset_edited,
                          batch_size=BATCH_SIZE)
results = AlexNet_edited.evaluate(test_dataset_edited, verbose=1)

fig = plt.figure(figsize=(15, 5))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

fig.suptitle('AlexNet edited images 448x448')
ax1.plot(hist.history['loss'], color='blue', label='loss', )
ax1.plot(hist.history['val_loss'], color='green', label='val_loss')
ax2.plot(hist.history['accuracy'], color='orange', label='accuracy')
ax2.plot(hist.history['val_accuracy'], color='red', label='val_accuracy')

plt.legend(loc="upper left")
plt.show()